In [1]:
import ast
import pandas as pd

import matplotlib.pyplot as plt

import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
new = pd.read_csv("data/socec_data.csv")

In [3]:
new.shape

(1219, 230)

In [38]:
new.iso_prov

0        0
1        1
2        2
3        3
4        4
        ..
1214    48
1215    49
1216    50
1217    51
1218    52
Name: iso_prov, Length: 1219, dtype: int64

## Script para la seleccion de variables 

In [44]:
import pandas as pd


FILENAME = "data/socec_data.csv"


def get_data(iso_prov, dt, socec_df=pd.read_csv(FILENAME)):
    """
    Get the closest to date socioeconomic data value available

    :param iso_prov: iso code of spanish subregion
    :param dt: date

    :return: pandas.Serie
    """

    yr = dt.year
    socec_df = socec_df[socec_df["iso_prov"] == iso_prov]

    to_return = pd.DataFrame({"year": [yr]})
    request_col = [col for col in socec_df.columns if col not in
                   to_return.columns]

    while request_col and (yr >= socec_df.year.min()):
        to_return = pd.concat([to_return.reset_index(drop=True),
                               socec_df.loc[socec_df["year"] == yr, :] \
                                       .loc[:, request_col] \
                                       .reset_index(drop=True)
                               ], axis=1) \
                      .dropna(axis=1, how="all")
        yr -= 1
        request_col = [col for col in socec_df.columns if col not in
                       to_return.columns]
    else:
        if request_col:
            #print(iso_prov, request_col, dt.year, sep=" / ")
            pass

    return pd.concat([pd.DataFrame(columns=socec_df.columns),
                      to_return], axis=0)


if __name__ == '__main__':
    from datetime import date

    print(get_data(39, date(2019, 2, 28)))
    print(get_data(8, date(2022, 1, 1)))


  iso_prov  year empresas.0 empresas.1 empresas.2 empresas.3 empresas.4  \
0       39  2019      38702        629      10496          1          0   

  empresas.5 empresas.6 empresas.7 empresas.8 empresas.9 sociedades.10.0.14  \
0       1027        115       2011        125      24298              642.0   

  sociedades.10.0.15 sociedades.10.0.16 sociedades.10.11.14  \
0            24272.0            24272.0                 1.0   

  sociedades.10.11.15 sociedades.10.11.16 sociedades.10.12.14  \
0                60.0                60.0               639.0   

  sociedades.10.12.15 sociedades.10.12.16 sociedades.10.13.14  \
0             24062.0             24062.0                 2.0   

  sociedades.10.13.15 sociedades.10.13.16 precio_trabajo.17 precio_trabajo.18  \
0               150.0               150.0           107.435               2.9   

  paro.19.0 paro.19.22 paro.19.23 paro.19.24 paro.19.25 paro.20.0 paro.20.22  \
0      12.2        7.0       19.4       21.6       11.2   

In [8]:
p = get_data(8, date(2022, 1, 1))

In [26]:
df = pd.read_csv("data/nuevoMutua.csv", dtype=str, header=[0, 1], index_col=0)


convert_dict = {
    ('casos_cc', 'duracion'): float, 
    ('casos_cc', 'dur_previo'): float, 
    ('casos_cc', 'dur_std'): float, 
    ('casos_cc', 'v_std'): float,
    ('casos_ep', 'nrehab'): float, 
    ('casos_ep', 'nbio'): float,
    ('aut', 'base_coti'): float, 
    ('creadas', 'edad'): float, 
    ('creadas', 'naltas'): float,
    ('creadas', 'next_fest'): float, 
    ('creadas', 'past_fest'): float,
    ('creadas', 'nfest_14'): float, 
    ('creadas', 'peso'): float, 
    ('creadas', 'dist_ult_baja'): float,
    ('creadas', 'avg_altas'): float,
    ('creadas', 'alta_oficio'): float,
    ('creadas', 'nrecaidas'): float,
    ('creadas', 'ncasos'): float,
    ('creadas', 'diff_ant_caso'): float,
    ('creadas', 'diff_ant_med_adm'): float,
    ('creadas', 'mean_ant_med_adm'): float,
    ('creadas', 'prev_rechazado'): float

}
df = df.astype(convert_dict)

date_cols = [
    ('casos_cc', 'f_parte'), 
    ('casos_cc', 'f_graba'), 
    ('casos_cc', 'f_baja'), 
    ('casos_cc', 'f_alta_adm'), 
    ('casos_cc', 'f_alta_med'), 
    ('casos_cc', 'f_susp_ext'),
    ('casos_ep', 'f_atrium'), 
    ('casos_ep', 'f_resp_atrium'), 
    ('casos_ep', 'f_ini_pd'), 
    ('aut', 'FECHA_AT'), 
    ('aut', 'FECHA_OPCION'), 
    ('aut', 'FECHA_MUTUA_IT'), 
    ('aut', 'f_alta_o_ss'), 
    ('aut', 'f_cese'), ('aut_fijo', 'f_nacm'),
    ('creadas', 'f_alta'), 
    ('creadas', 'f_baja')

]
for col in date_cols:
    df[col] = pd.to_datetime(df[col], format='%Y-%m-%d').dt.date


df[('aut', 'HORA_PRE')] = pd.to_datetime(df[('aut', 'HORA_PRE')], format='%H:%M:%S').dt.time
def tablas_en_csv(csv,*, verbose=False):
    '''
    Funcion para visualiar el numero de tanlas en el dataframe 
    
    Retorna los nombres de las tablas
    '''
    llaves = list(csv.keys())
    if type(llaves[0]) == tuple:
        llaves = np.array(llaves)
        tablas = set(llaves[:,0])
        if verbose:
            print(f'Hay {len(tablas)} tablas')
        return list(tablas)
    else:
        print(f'El dataset contiene una unica tabla')

tablas = tablas_en_csv(df,verbose=True)
tablas

tablasdf = dict()
for i in tablas:
    tablasdf[i] = df[i]

df = pd.concat(tablasdf.values(), axis=1)

df.shape

Hay 7 tablas


(10018, 111)

In [46]:
iso = df.provincia.astype("int")


date = df.loc[:,"f_parte"]

In [47]:
newData = []
for i, d in zip(iso, date):
    newData.append(get_data(i,d))

In [48]:
hola = pd.concat(newData)

In [49]:
hola.head()

,iso_prov,year,empresas.0,empresas.1,empresas.2,empresas.3,empresas.4,empresas.5,empresas.6,empresas.7,empresas.8,empresas.9,sociedades.10.0.14,sociedades.10.0.15,sociedades.10.0.16,sociedades.10.11.14,sociedades.10.11.15,sociedades.10.11.16,sociedades.10.12.14,sociedades.10.12.15,sociedades.10.12.16,sociedades.10.13.14,sociedades.10.13.15,sociedades.10.13.16,precio_trabajo.17,precio_trabajo.18,paro.19.0,paro.19.22,paro.19.23,paro.19.24,paro.19.25,paro.20.0,paro.20.22,paro.20.23,paro.20.24,paro.20.25,paro.21.0,paro.21.22,paro.21.23,paro.21.24,paro.21.25,actividad.19.0,actividad.19.22,actividad.19.23,actividad.19.24,actividad.19.25,actividad.20.0,actividad.20.22,actividad.20.23,actividad.20.24,actividad.20.25,actividad.21.0,actividad.21.22,actividad.21.23,actividad.21.24,actividad.21.25,ocupados.0.0,ocupados.0.22,ocupados.0.23,ocupados.0.24,ocupados.0.25,ocupados.26.0,ocupados.26.22,ocupados.26.23,ocupados.26.24,ocupados.26.25,ocupados.27.0,ocupados.27.22,ocupados.27.23,ocupados.27.24,ocupados.27.25,ocupados.28.0,ocupados.28.22,ocupados.28.23,ocupados.28.24,ocupados.28.25,ocupados.29.0,ocupados.29.22,ocupados.29.23,ocupados.29.24,ocupados.29.25,d_salarial.19.30,d_salarial.19.31,d_salarial.19.32,d_salarial.19.33,d_salarial.19.34,d_salarial.19.35,d_salarial.20.30,d_salarial.20.31,d_salarial.20.32,d_salarial.20.33,d_salarial.20.34,d_salarial.20.35,d_salarial.21.30,d_salarial.21.31,d_salarial.21.32,d_salarial.21.33,d_salarial.21.34,d_salarial.21.35,gat_sector.36.19,gat_sector.36.20,gat_sector.36.21,gat_sector.37.19,gat_sector.37.20,gat_sector.37.21,gat_sector.38.19,gat_sector.38.20,gat_sector.38.21,gat_sector.39.19,gat_sector.39.20,gat_sector.39.21,gat_ocupacion.30.19,gat_ocupacion.30.20,gat_ocupacion.30.21,gat_ocupacion.40.19,gat_ocupacion.40.20,gat_ocupacion.40.21,gat_ocupacion.41.19,gat_ocupacion.41.20,gat_ocupacion.41.21,gat_ocupacion.42.19,gat_ocupacion.42.20,gat_ocupacion.42.21,gat_contrato.0.19,gat_contrato.0.20,gat_contrato.0.21,gat_contrato.43.19,gat_contrato.43.20,gat_contrato.43.21,gat_contrato.44.19,gat_contrato.44.20,gat_contrato.44.21,ght_sector.36.19,ght_sector.36.20,ght_sector.36.21,ght_sector.37.19,ght_sector.37.20,ght_sector.37.21,ght_sector.38.19,ght_sector.38.20,ght_sector.38.21,ght_sector.39.19,ght_sector.39.20,ght_sector.39.21,ght_ocupacion.30.19,ght_ocupacion.30.20,ght_ocupacion.30.21,ght_ocupacion.40.19,ght_ocupacion.40.20,ght_ocupacion.40.21,ght_ocupacion.41.19,ght_ocupacion.41.20,ght_ocupacion.41.21,ght_ocupacion.42.19,ght_ocupacion.42.20,ght_ocupacion.42.21,ght_contrato.0.19,ght_contrato.0.20,ght_contrato.0.21,ght_contrato.43.19,ght_contrato.43.20,ght_contrato.43.21,ght_contrato.44.19,ght_contrato.44.20,ght_contrato.44.21,renta_p.45,renta_p.46,renta_p.47,renta_p.48,renta_h.49,renta_h.50,renta_d.51,renta_d.52,renta_d.53,renta_d.54,renta_d.55,renta_d.56,renta_d.57,renta_d.58,renta_d.59,renta_d.60,pobreza.62,pobreza.63,carencia_p.64,carencia_p.65,carencia_p.66,carencia_p.67,carencia_p.68,carencia_p.69,carencia_p.70,carencia_h.64,carencia_h.65,carencia_h.66,carencia_h.67,carencia_h.68,carencia_h.69,carencia_h.70,ncarencia_p.71,ncarencia_p.72,ncarencia_p.73,ncarencia_h.71,ncarencia_h.72,ncarencia_h.73,p_dificultades.61,p_dificultades.74,p_dificultades.75,p_dificultades.76,p_dificultades.77,p_dificultades.78,p_dificultades.79,h_dificultades.61,h_dificultades.74,h_dificultades.75,h_dificultades.76,h_dificultades.77,h_dificultades.78,h_dificultades.79,rvivienda_h.80,rvivienda_h.81,rvivienda_h.82,rvivienda_h.83,pvivienda_h.84,pvivienda_h.85,pvivienda_h.86,pvivienda_h.87,pvivienda_h.88,pobreza_exclusion.89,pobreza_exclusion.90,pobreza_exclusion.91,pobreza_exclusion.92
0,8,2019,470067,12281,166934,15,18,15197,2015,24249,888,248470,14695.0,983103.0,981662.0,63.0,34104.0,32663.0,14631.0,948989.0,948989.0,1.0,10.0,10.0,106.756,0.5,11.6,15.1,14.4,14.8,10.9,11.1,18.7,15.1,10.2,10.4,12.2,11.7,13.6,19.9,11.5,61.4,71.7,75.3,75.0,58.9,66.3,79.2,80.5,81.0,63.6,56.7,65.9,69.9,69.1,54.5,3391.0,132.3,281.8,220.0,2756.9,1814.7,86

In [51]:
hola = hola.drop(["iso_prov","year"], axis=1)

In [55]:
hola = hola.reset_index().drop("index", axis=1)

In [57]:
hola.to_csv("data/socec_data_fil.csv")

In [63]:
hola.isna().sum().value_counts()

8       226
7551      2
dtype: int64